# Домашняя работа к девятому семинару

Использование алгоритмов понижения размерности для улучшения классификации новостей (https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html)

Цель задания: Исследовать влияние различных методов понижения размерности на качество классификации текстовых данных.

Датасет: Набор данных новостных статей
(датасет '20 Newsgroups' доступный в sklearn.datasets).

Задачи:

1. Загрузите датасет '20 Newsgroups' из sklearn.

2. Проведите предобработку данных (очистка текста, удаление стоп-слов, векторизация с использованием TF-IDF).

3. Примените к полученным векторам TF-IDF следующие методы понижения размерности:
 - PCA (Principal Component Analysis)
 - t-SNE (t-distributed Stochastic Neighbor Embedding)
 - UMAP (Uniform Manifold Approximation and Projection).

4. После понижения размерности данных используйте любой метод машинного обучения для классификации новостей по темам.

5. Сравните качество классификации для каждого метода понижения размерности. Используйте метрики точности и F1-меру.

6. Визуализируйте двумерное представление данных для каждого метода понижения размерности, чтобы оценить, как алгоритмы справляются с сепарацией классов.

7. Напишите отчёт, в котором обсудите, какой метод понижения размерности оказал наиболее значительное влияние на качество классификации и почему.

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA, TruncatedSVD, IncrementalPCA
from sklearn.manifold import TSNE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
import umap

## 1. Загрузка датасета

In [2]:
newsgroups = fetch_20newsgroups(subset='all')

## 2. Предобработка данных

In [3]:
# Очистка текста и удаление стоп-слов
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(newsgroups.data)
y = newsgroups.target

In [4]:
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 3. Применение методов понижения размерности

In [8]:
# # PCA
# pca = PCA(n_components=2)
# X_train_pca = pca.fit_transform(X_train.toarray())
# X_test_pca = pca.transform(X_test.toarray())

При попытке преобразовать разрежённую матрицу в плотную с помощью метода toarray() возникла ошибка `MemoryError`. Ошибка `MemoryError` возникает, когда системе не хватает памяти для выполнения операции. В данном случае, проблема связана с попыткой преобразовать разреженную матрицу в плотную, что требует значительного объема памяти.

Попробуем решить эту проблему за счёт использования разреженных матриц: Вместо преобразования разреженной матрицы в плотную, используем методы, которые работают непосредственно с разреженными матрицами. Например, можно использовать TruncatedSVD из библиотеки scikit-learn, который работает с разреженными матрицами:

In [ ]:
svd = TruncatedSVD(n_components=2)
X_train_svd = svd.fit_transform(X_train)
X_test_svd = svd.transform(X_test)

In [10]:
# Предположим, что X_train - это разреженная матрица
batch_size = 1000  # Размер пакета
n_components = 2  # Количество компонент PCA

ipca = IncrementalPCA(n_components=n_components)

# Обработка данных пакетами
for i in range(0, X_train.shape[0], batch_size):
    X_batch = X_train[i:i + batch_size].toarray()
    ipca.partial_fit(X_batch)

# Преобразование данных
X_train_pca = np.vstack([ipca.transform(X_train[i:i + batch_size].toarray()) for i in range(0, X_train.shape[0], batch_size)])
X_test_pca = np.vstack([ipca.transform(X_test[i:i + batch_size].toarray()) for i in range(0, X_test.shape[0], batch_size)])


In [13]:
# Предположим, что X_train - это разреженная матрица
batch_size = 1000  # Размер пакета
n_components = 2  # Количество компонент PCA

ipca = IncrementalPCA(n_components=n_components)

# Обработка данных пакетами с прогрессбаром
for i in tqdm(range(0, X_train.shape[0], batch_size), desc="Training PCA"):
    X_batch = X_train[i:i + batch_size].toarray()
    ipca.partial_fit(X_batch)

# Преобразование данных с прогрессбаром
X_train_pca = np.vstack([ipca.transform(X_train[i:i + batch_size].toarray()) for i in tqdm(range(0, X_train.shape[0], batch_size), desc="Transforming Train Data")])
X_test_pca = np.vstack([ipca.transform(X_test[i:i + batch_size].toarray()) for i in tqdm(range(0, X_test.shape[0], batch_size), desc="Transforming Test Data")])


Transforming Test Data: 100%|██████████| 4/4 [00:03<00:00,  1.04it/s]


In [14]:
# t-SNE
tsne = TSNE(n_components=2, random_state=42)
X_train_tsne = tsne.fit_transform(X_train.toarray())
X_test_tsne = tsne.transform(X_test.toarray())

MemoryError: Unable to allocate 19.5 GiB for an array with shape (15076, 173451) and data type float64

In [15]:
# UMAP
umap_reducer = umap.UMAP(n_components=2, random_state=42)
X_train_umap = umap_reducer.fit_transform(X_train.toarray())
X_test_umap = umap_reducer.transform(X_test.toarray())

MemoryError: Unable to allocate 19.5 GiB for an array with shape (15076, 173451) and data type float64

## 4. Классификация новостей

Для классификации можно использовать, например, логистическую регрессию:

In [ ]:
# Функция для обучения и оценки модели
def evaluate_model(X_train, X_test, y_train, y_test):
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    return accuracy, f1

In [ ]:
# Оценка моделей
accuracy_pca, f1_pca = evaluate_model(X_train_pca, X_test_pca, y_train, y_test)
accuracy_tsne, f1_tsne = evaluate_model(X_train_tsne, X_test_tsne, y_train, y_test)
accuracy_umap, f1_umap = evaluate_model(X_train_umap, X_test_umap, y_train, y_test)

## 5. Сравнение качества классификации

In [ ]:
print(f"PCA: Accuracy = {accuracy_pca}, F1 Score = {f1_pca}")
print(f"t-SNE: Accuracy = {accuracy_tsne}, F1 Score = {f1_tsne}")
print(f"UMAP: Accuracy = {accuracy_umap}, F1 Score = {f1_umap}")

## 6. Визуализация данных

In [ ]:
def plot_2d(X, y, title):
    plt.figure(figsize=(10, 7))
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap='viridis', s=5)
    plt.colorbar()
    plt.title(title)
    plt.show()

In [ ]:
plot_2d(X_train_pca, y_train, 'PCA')
plot_2d(X_train_tsne, y_train, 't-SNE')
plot_2d(X_train_umap, y_train, 'UMAP')

## 7. Написание отчета

В отчете обсудите результаты, сравните метрики точности и F1-меры для каждого метода понижения размерности, а также визуализации. Обратите внимание на то, какой метод лучше справляется с сепарацией классов и почему.